In [1]:
import sys
sys.path.append('..')
import logging
import rtde.rtde as rtde
import rtde.rtde_config as rtde_config
import time
import math

In [2]:
#logging.basicConfig(level=logging.INFO)

ROBOT_HOST = '192.168.188.123'
ROBOT_PORT = 30004
config_filename = 'pose_pilot_configuration.xml'

keep_running = True
logging.getLogger().setLevel(logging.INFO)

In [3]:
conf = rtde_config.ConfigFile(config_filename)
state_names, state_types = conf.get_recipe('state')
setp_names, setp_types = conf.get_recipe('setp')
watchdog_names, watchdog_types = conf.get_recipe('watchdog')

In [4]:
con = rtde.RTDE(ROBOT_HOST, ROBOT_PORT)
con.connect()

# get controller version
con.get_controller_version()

# setup recipes
con.send_output_setup(state_names, state_types)
setp = con.send_input_setup(setp_names, setp_types)
watchdog = con.send_input_setup(watchdog_names, watchdog_types)

INFO:root:Controller version: 3.4.3.386


In [ ]:
# Hyper parameters
T = 30 #sec, Time constant
START_TIME = time.time()
TOL_DIST = [0.05, 0.05, 0.05, 0.02, 0.02, 0.02] # The max target distance from current pose in each step

In [5]:
# Setpoints to move the robot to
setp1 = [0.79, -0.51, 0.97, 0.57, -2.16, 1.68]
setp2 = [0.32, -0.41, 1.02, 0.57, -2.16, 1.68]

center = [(a + b)/2 for a, b in zip(setp1, setp2)]
amp = [(b - a)/2 for a, b in zip(setp1, setp2)]

setp.input_double_register_0 = 0
setp.input_double_register_1 = 0
setp.input_double_register_2 = 0
setp.input_double_register_3 = 0
setp.input_double_register_4 = 0
setp.input_double_register_5 = 0
  
# The function "rtde_set_watchdog" in the "rtde_control_loop.urp" creates a 1 Hz watchdog
watchdog.input_int_register_0 = 0

In [6]:
def setp_to_list(setp):
    list = []
    for i in range(0,6):
        list.append(setp.__dict__["input_double_register_%i" % i])
    return list

def list_to_setp(setp, list):
    for i in range (0,6):
        setp.__dict__["input_double_register_%i" % i] = list[i]
    return setp
def correct_target_inf(target, current, bound):
    # Returns target location within bounding box
    # of current location.
    # target, current and bound are lists
    

In [7]:
#start data synchronization
if not con.send_start():
    sys.exit()

# control loop
while keep_running:
    # receive the current state
    state = con.receive()
    
    if state is None:
        break;
    
    # Calc new setp
    current_rad = 2*math.pi*(time.time() - START_TIME)/T
    target_pose = [c + a*math.sin(current_rad) for a,c in zip(amp, center)]
    
    # Put new targetp in the buffer
    # If buffer is not empty read from buffer.
    
    # We have assumed infinite norm bounding box is considered to give error when using servoj command
    # TODO: Check if inf norm is correct assumption
    correct_target_inf(target_pose, state.target_q, TOL_DIST)
    
    list_to_setp(setp, new_setp)
    
    # send new setpoint        
    con.send(setp)
    
    # kick watchdog
    con.send(watchdog)

INFO:root:RTDE synchronization started


error: [Errno 54] Connection reset by peer

In [ ]:
con.send_pause()
con.disconnect()